In [1]:
import pyspark

#Creamos una sesion de spark
from pyspark.sql import SparkSession

#Creamos un objeto de spark session
spark = SparkSession.builder.appName("pysparkdf").getOrCreate()

In [2]:
#Leemos un cvs
df_2021 = spark.read.format("csv").option("header", "true").option("delimiter", ",").load("C:/Users/USER/Desktop/DayanaFrepost/big-data-processing/Proyecto-Final/Datasets/world-happiness-report-2021.csv")


In [3]:
#Para ver el esquema de los datos:
df_2021.printSchema()

root
 |-- Country name: string (nullable = true)
 |-- Regional indicator: string (nullable = true)
 |-- Ladder score: string (nullable = true)
 |-- Standard error of ladder score: string (nullable = true)
 |-- upperwhisker: string (nullable = true)
 |-- lowerwhisker: string (nullable = true)
 |-- Logged GDP per capita: string (nullable = true)
 |-- Social support: string (nullable = true)
 |-- Healthy life expectancy: string (nullable = true)
 |-- Freedom to make life choices: string (nullable = true)
 |-- Generosity: string (nullable = true)
 |-- Perceptions of corruption: string (nullable = true)
 |-- Ladder score in Dystopia: string (nullable = true)
 |-- Explained by: Log GDP per capita: string (nullable = true)
 |-- Explained by: Social support: string (nullable = true)
 |-- Explained by: Healthy life expectancy: string (nullable = true)
 |-- Explained by: Freedom to make life choices: string (nullable = true)
 |-- Explained by: Generosity: string (nullable = true)
 |-- Explained 

In [4]:
#Seleccionamos ciertas columnas
df_2021.select('Country name', 'Ladder score','Regional indicator').show(10)

+------------+------------+--------------------+
|Country name|Ladder score|  Regional indicator|
+------------+------------+--------------------+
|     Finland|       7.842|      Western Europe|
|     Denmark|       7.620|      Western Europe|
| Switzerland|       7.571|      Western Europe|
|     Iceland|       7.554|      Western Europe|
| Netherlands|       7.464|      Western Europe|
|      Norway|       7.392|      Western Europe|
|      Sweden|       7.363|      Western Europe|
|  Luxembourg|       7.324|      Western Europe|
| New Zealand|       7.277|North America and...|
|     Austria|       7.268|      Western Europe|
+------------+------------+--------------------+
only showing top 10 rows



In [5]:
from pyspark.sql.functions import filter,first


In [6]:
#isNotNull()
from pyspark.sql.functions import *



In [7]:
from pyspark.sql.functions import desc


In [8]:
df_2021_ordenado = df_2021.select('Country name', 'Ladder score','Regional indicator').orderBy(desc("Ladder score"), desc("Country name"))
# Muestra el DataFrame ordenado
df_2021_ordenado.show()

+--------------+------------+--------------------+
|  Country name|Ladder score|  Regional indicator|
+--------------+------------+--------------------+
|       Finland|       7.842|      Western Europe|
|       Denmark|       7.620|      Western Europe|
|   Switzerland|       7.571|      Western Europe|
|       Iceland|       7.554|      Western Europe|
|   Netherlands|       7.464|      Western Europe|
|        Norway|       7.392|      Western Europe|
|        Sweden|       7.363|      Western Europe|
|    Luxembourg|       7.324|      Western Europe|
|   New Zealand|       7.277|North America and...|
|       Austria|       7.268|      Western Europe|
|     Australia|       7.183|North America and...|
|        Israel|       7.157|Middle East and N...|
|       Germany|       7.155|      Western Europe|
|        Canada|       7.103|North America and...|
|       Ireland|       7.085|      Western Europe|
|    Costa Rica|       7.069|Latin America and...|
|United Kingdom|       7.064|  

In [9]:
# *******Pregunta 1: ¿Cuál es el país más “feliz” del 2021 según la data? (considerar que la columna “Ladder score” mayor número más feliz es el país)
happiest_country = df_2021.first()["Country name"]

print("El país más feliz del 2021 es:", happiest_country)

El país más feliz del 2021 es: Finland


In [10]:
#*******Pregunta 2: ¿Cuál es el país más “feliz” del 2021 por continente según la data?

   #Agrupar por Regional Indicator
grouped_df_2021 = df_2021_ordenado.groupBy("Regional indicator")


   # Ordenar cada grupo según la columna "Ladder score" en orden descendente
sorted_df_2021 = grouped_df_2021.agg(
    first("Country name").alias("Country name"), 
    first("Ladder score").alias("Ladder score")
).orderBy(desc("Ladder score"))

   # Imprimir el país más feliz por continente
sorted_df_2021.show()

+--------------------+--------------------+------------+
|  Regional indicator|        Country name|Ladder score|
+--------------------+--------------------+------------+
|      Western Europe|             Finland|       7.842|
|North America and...|         New Zealand|       7.277|
|Middle East and N...|              Israel|       7.157|
|Latin America and...|          Costa Rica|       7.069|
|Central and Easte...|      Czech Republic|       6.965|
|           East Asia|Taiwan Province o...|       6.584|
|      Southeast Asia|           Singapore|       6.377|
|Commonwealth of I...|          Uzbekistan|       6.179|
|  Sub-Saharan Africa|           Mauritius|       6.049|
|          South Asia|               Nepal|       5.269|
+--------------------+--------------------+------------+



In [11]:
#¿Cuál es el país que más veces ocupó el primer lugar en todos los años?
#leemos otra vez 2021
df_2021 = spark.read.format("csv").option("header", "true").option("delimiter", ",").load("C:/Users/USER/Desktop/DayanaFrepost/big-data-processing/Proyecto-Final/Datasets/world-happiness-report-2021.csv")
df_2021.printSchema()



root
 |-- Country name: string (nullable = true)
 |-- Regional indicator: string (nullable = true)
 |-- Ladder score: string (nullable = true)
 |-- Standard error of ladder score: string (nullable = true)
 |-- upperwhisker: string (nullable = true)
 |-- lowerwhisker: string (nullable = true)
 |-- Logged GDP per capita: string (nullable = true)
 |-- Social support: string (nullable = true)
 |-- Healthy life expectancy: string (nullable = true)
 |-- Freedom to make life choices: string (nullable = true)
 |-- Generosity: string (nullable = true)
 |-- Perceptions of corruption: string (nullable = true)
 |-- Ladder score in Dystopia: string (nullable = true)
 |-- Explained by: Log GDP per capita: string (nullable = true)
 |-- Explained by: Social support: string (nullable = true)
 |-- Explained by: Healthy life expectancy: string (nullable = true)
 |-- Explained by: Freedom to make life choices: string (nullable = true)
 |-- Explained by: Generosity: string (nullable = true)
 |-- Explained 

In [12]:
#Leemos un cvs anteriores a 2021
df_anterior = spark.read.format("csv").option("header", "true").option("delimiter", ",").load("C:/Users/USER/Desktop/DayanaFrepost/big-data-processing/Proyecto-Final/Datasets/world-happiness-report.csv")
df_anterior.printSchema()

root
 |-- Country name: string (nullable = true)
 |-- year: string (nullable = true)
 |-- Life Ladder: string (nullable = true)
 |-- Log GDP per capita: string (nullable = true)
 |-- Social support: string (nullable = true)
 |-- Healthy life expectancy at birth: string (nullable = true)
 |-- Freedom to make life choices: string (nullable = true)
 |-- Generosity: string (nullable = true)
 |-- Perceptions of corruption: string (nullable = true)
 |-- Positive affect: string (nullable = true)
 |-- Negative affect: string (nullable = true)



In [13]:
df_anterior_ordenado = df_anterior.select('year','Country name', 'Life Ladder').orderBy(desc("Life Ladder"), desc("Country name"))
# Muestra el DataFrame ordenado
df_anterior_ordenado.show()

+----+------------+-----------+
|year|Country name|Life Ladder|
+----+------------+-----------+
|2005|     Denmark|      8.019|
|2008|     Denmark|      7.971|
|2020|     Finland|      7.889|
|2018|     Finland|      7.858|
|2007|     Denmark|      7.834|
|2017|     Finland|      7.788|
|2011|     Denmark|      7.788|
|2019|     Finland|      7.780|
|2012| Switzerland|      7.776|
|2010|     Denmark|      7.771|
|2019| Switzerland|      7.694|
|2019|     Denmark|      7.693|
|2009|     Denmark|      7.683|
|2012|      Norway|      7.678|
|2006|     Finland|      7.672|
|2008|     Finland|      7.671|
|2016|     Finland|      7.660|
|2010|      Canada|      7.650|
|2018|     Denmark|      7.649|
|2008|      Norway|      7.632|
+----+------------+-----------+
only showing top 20 rows



In [14]:
from pyspark.sql.functions import lit

# Agregar columna year con contenido "2021"
df_2021_with_year = df_2021.withColumn("year", lit("2021"))
df_2021_with_year.printSchema()

root
 |-- Country name: string (nullable = true)
 |-- Regional indicator: string (nullable = true)
 |-- Ladder score: string (nullable = true)
 |-- Standard error of ladder score: string (nullable = true)
 |-- upperwhisker: string (nullable = true)
 |-- lowerwhisker: string (nullable = true)
 |-- Logged GDP per capita: string (nullable = true)
 |-- Social support: string (nullable = true)
 |-- Healthy life expectancy: string (nullable = true)
 |-- Freedom to make life choices: string (nullable = true)
 |-- Generosity: string (nullable = true)
 |-- Perceptions of corruption: string (nullable = true)
 |-- Ladder score in Dystopia: string (nullable = true)
 |-- Explained by: Log GDP per capita: string (nullable = true)
 |-- Explained by: Social support: string (nullable = true)
 |-- Explained by: Healthy life expectancy: string (nullable = true)
 |-- Explained by: Freedom to make life choices: string (nullable = true)
 |-- Explained by: Generosity: string (nullable = true)
 |-- Explained 

In [15]:
#renombro la columna ladder a life ladder
df_2021_with_year = df_2021_with_year.withColumnRenamed("Ladder score", "Life Ladder")
df_2021_with_year.select('year','Country name','Life Ladder').show(20)


+----+--------------+-----------+
|year|  Country name|Life Ladder|
+----+--------------+-----------+
|2021|       Finland|      7.842|
|2021|       Denmark|      7.620|
|2021|   Switzerland|      7.571|
|2021|       Iceland|      7.554|
|2021|   Netherlands|      7.464|
|2021|        Norway|      7.392|
|2021|        Sweden|      7.363|
|2021|    Luxembourg|      7.324|
|2021|   New Zealand|      7.277|
|2021|       Austria|      7.268|
|2021|     Australia|      7.183|
|2021|        Israel|      7.157|
|2021|       Germany|      7.155|
|2021|        Canada|      7.103|
|2021|       Ireland|      7.085|
|2021|    Costa Rica|      7.069|
|2021|United Kingdom|      7.064|
|2021|Czech Republic|      6.965|
|2021| United States|      6.951|
|2021|       Belgium|      6.834|
+----+--------------+-----------+
only showing top 20 rows



In [16]:
df_completo = df_anterior_ordenado.union(df_2021_with_year.select('year','Country name','Life Ladder'))
df_completo = df_completo.orderBy(desc("Life Ladder"), desc("Country name"))
# Mostrar el resultado
df_completo.show()

+----+------------+-----------+
|year|Country name|Life Ladder|
+----+------------+-----------+
|2005|     Denmark|      8.019|
|2008|     Denmark|      7.971|
|2020|     Finland|      7.889|
|2018|     Finland|      7.858|
|2021|     Finland|      7.842|
|2007|     Denmark|      7.834|
|2017|     Finland|      7.788|
|2011|     Denmark|      7.788|
|2019|     Finland|      7.780|
|2012| Switzerland|      7.776|
|2010|     Denmark|      7.771|
|2019| Switzerland|      7.694|
|2019|     Denmark|      7.693|
|2009|     Denmark|      7.683|
|2012|      Norway|      7.678|
|2006|     Finland|      7.672|
|2008|     Finland|      7.671|
|2016|     Finland|      7.660|
|2010|      Canada|      7.650|
|2018|     Denmark|      7.649|
+----+------------+-----------+
only showing top 20 rows



In [17]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, row_number

# Definir la ventana por la cual se ordenará el dataset y se obtendrá el ranking
window = Window.partitionBy('year').orderBy(desc('Life Ladder'))

# Agregar una columna 'ranking' con el ranking por año de los países por Life Ladder
df_ranking = df_completo.withColumn('ranking', row_number().over(window)).where('ranking == 1')

# Seleccionar las columnas deseadas y mostrar el resultado
df_resultado = df_ranking.select('year', 'Country name', 'Life Ladder')
df_resultado.show()


+----+------------+-----------+
|year|Country name|Life Ladder|
+----+------------+-----------+
|2005|     Denmark|      8.019|
|2006|     Finland|      7.672|
|2007|     Denmark|      7.834|
|2008|     Denmark|      7.971|
|2009|     Denmark|      7.683|
|2010|     Denmark|      7.771|
|2011|     Denmark|      7.788|
|2012| Switzerland|      7.776|
|2013|      Canada|      7.594|
|2014|     Denmark|      7.508|
|2015|      Norway|      7.603|
|2016|     Finland|      7.660|
|2017|     Finland|      7.788|
|2018|     Finland|      7.858|
|2019|     Finland|      7.780|
|2020|     Finland|      7.889|
|2021|     Finland|      7.842|
+----+------------+-----------+



In [18]:
#******Pregunta 3:¿Cuál es el país que más veces ocupó el primer lugar en todos los años?
from pyspark.sql.functions import desc

# Agrupar por país, contar cuántas veces aparece cada país y ordenar en orden descendente
repeticiones_pais = df_resultado.groupBy('Country name').count().orderBy(('count'))

# Imprimir el resultado
repeticiones_pais.show()


+------------+-----+
|Country name|count|
+------------+-----+
|      Canada|    1|
| Switzerland|    1|
|      Norway|    1|
|     Finland|    7|
|     Denmark|    7|
+------------+-----+



In [19]:
from pyspark.sql.functions import col

# Convertir columnas numéricas a DoubleType
num_cols = ['Life Ladder', 'Log GDP per capita', 'Social support',             'Healthy life expectancy at birth', 'Freedom to make life choices',            'Generosity', 'Perceptions of corruption', 'Positive affect', 'Negative affect']

for col_name in num_cols:
    df_anterior = df_anterior.withColumn(col_name, col(col_name).cast('double'))

# Filtrar datos del año 2020
df_2020 = df_anterior.filter(col('year') == '2020')

# Ordenar por Log GDP per capita descendente y seleccionar columnas relevantes
top_2020 = df_2020.select('Country name', 'Log GDP per capita','Life Ladder') \
                .orderBy(col('Log GDP per capita').desc()) \
        

# Mostrar resultados
top_2020.show()


+--------------------+------------------+-----------+
|        Country name|Log GDP per capita|Life Ladder|
+--------------------+------------------+-----------+
|             Ireland|            11.323|      7.035|
|         Switzerland|            11.081|      7.508|
|United Arab Emirates|            11.053|      6.458|
|              Norway|            11.042|       7.29|
|       United States|            11.001|      7.028|
|             Denmark|             10.91|      7.515|
|         Netherlands|            10.901|      7.504|
|             Austria|            10.851|      7.213|
|              Sweden|            10.838|      7.314|
|             Germany|            10.833|      7.312|
|             Iceland|            10.824|      7.575|
|             Belgium|            10.771|      6.839|
|           Australia|             10.76|      7.137|
|             Finland|             10.75|      7.889|
|              Canada|             10.73|      7.025|
|        Saudi Arabia|      

In [ ]:
# Pregunta No.4. ¿Qué puesto de Felicidad tiene el país con mayor GDP del 2020?

In [20]:
from pyspark.sql.functions import row_number, desc
from pyspark.sql.window import Window

top_2020 = df_2020.select('Country name', 'Log GDP per capita', 'Life Ladder') \
                  .orderBy(desc('Log GDP per capita'))

window_gdp = Window.orderBy(desc('Log GDP per capita'))
window_ladder = Window.orderBy(desc('Life Ladder'))
top_2020 = top_2020.withColumn('Rank Life Ladder', row_number().over(window_ladder))
top_2020 = top_2020.withColumn('Rank GDP', row_number().over(window_gdp))

top_2020.show()

+--------------------+------------------+-----------+----------------+--------+
|        Country name|Log GDP per capita|Life Ladder|Rank Life Ladder|Rank GDP|
+--------------------+------------------+-----------+----------------+--------+
|             Ireland|            11.323|      7.035|              13|       1|
|         Switzerland|            11.081|      7.508|               4|       2|
|United Arab Emirates|            11.053|      6.458|              27|       3|
|              Norway|            11.042|       7.29|               8|       4|
|       United States|            11.001|      7.028|              14|       5|
|             Denmark|             10.91|      7.515|               3|       6|
|         Netherlands|            10.901|      7.504|               5|       7|
|             Austria|            10.851|      7.213|              10|       8|
|              Sweden|            10.838|      7.314|               6|       9|
|             Germany|            10.833

In [ ]:
#Pregunta No 5. ¿En que porcentaje ha variado a nivel mundial el GDP promedio del 2020 respecto al 2021? ¿Aumentó o disminuyó?

In [23]:
from pyspark.sql.functions import avg
df_2020 = df_2020.withColumnRenamed('Log GDP per capita', 'Logged GDP per capita')

# Seleccionar las columnas relevantes de cada dataset
df_2020_sel = df_2020.select('Country name', 'Logged GDP per capita')
df_2021_sel = df_2021_with_year.select('Country name', 'Logged GDP per capita')

# Unir los dos dataframes por el campo 'Country name'
df_join = df_2020_sel.join(df_2021_sel, ['Country name'])

# Calcular los valores promedio de Log GDP per capita en cada año
avg_gdp_2020 = df_2020_sel.agg(avg('Logged GDP per capita')).collect()[0][0]
avg_gdp_2021 = df_2021_sel.agg(avg('Logged GDP per capita')).collect()[0][0]

# Calcular la diferencia porcentual entre los valores promedio de Log GDP per capita
pct_change = ((avg_gdp_2021 - avg_gdp_2020) / avg_gdp_2020) * 100

# Imprimir el resultado
print("Promedio 2021:",avg_gdp_2021)
print("Promedio 2020:",avg_gdp_2020)
print(f'El Log GDP per capita promedio a nivel mundial ha variado en un {pct_change:.2f}% entre 2020 y 2021.')


Promedio 2021: 9.432208053691273
Promedio 2020: 9.751329545454546
El Log GDP per capita promedio a nivel mundial ha variado en un -3.27% entre 2020 y 2021.


In [ ]:
#¿Cuál es el país con mayor expectativa de vide (“Healthy life expectancy at birth”)? Y ¿Cuánto tenia en ese indicador en el 2019?

In [24]:
from pyspark.sql.functions import desc
df_anterior_healty= df_anterior.withColumnRenamed('Healthy life expectancy at birth', 'Healthy life expectancy')
max_life_expectancy = df_anterior_healty.select('Country name', 'Healthy life expectancy','year') \
                                .filter(df_anterior.year == '2019') \
                                .orderBy(desc('Healthy life expectancy')) \
                                .first()

print("El país con mayor expectativa de vida en 2019 fue", max_life_expectancy[0])
print("Tenía una expectativa de vida de", max_life_expectancy[1], "años.")


El país con mayor expectativa de vida en 2019 fue Singapore
Tenía una expectativa de vida de 77.1 años.


In [29]:
df_2021_with_year_healty=df_2021_with_year.select('Country name','Healthy life expectancy','year').show(20)

+--------------+-----------------------+----+
|  Country name|Healthy life expectancy|year|
+--------------+-----------------------+----+
|       Finland|                 72.000|2021|
|       Denmark|                 72.700|2021|
|   Switzerland|                 74.400|2021|
|       Iceland|                 73.000|2021|
|   Netherlands|                 72.400|2021|
|        Norway|                 73.300|2021|
|        Sweden|                 72.700|2021|
|    Luxembourg|                 72.600|2021|
|   New Zealand|                 73.400|2021|
|       Austria|                 73.300|2021|
|     Australia|                 73.900|2021|
|        Israel|                 73.503|2021|
|       Germany|                 72.500|2021|
|        Canada|                 73.800|2021|
|       Ireland|                 72.400|2021|
|    Costa Rica|                 71.400|2021|
|United Kingdom|                 72.500|2021|
|Czech Republic|                 70.807|2021|
| United States|                 6

In [25]:
df_completo_healty = df_anterior_healty.select('Country name', 'Healthy life expectancy','year').union(df_2021_with_year.select('Country name','Healthy life expectancy','year'))
df_completo_healty = df_completo_healty.orderBy(desc('Healthy life expectancy'), desc("Country name"))

# Mostrar el resultado
df_completo_healty.show()

+--------------------+-----------------------+----+
|        Country name|Healthy life expectancy|year|
+--------------------+-----------------------+----+
|           Singapore|                   77.1|2019|
|           Singapore|                 76.953|2021|
|Hong Kong S.A.R. ...|                 76.820|2021|
|           Singapore|                   76.8|2018|
|           Singapore|                   76.5|2017|
|           Singapore|                   76.2|2016|
|           Singapore|                   75.9|2015|
|           Singapore|                  75.68|2014|
|           Singapore|                  75.46|2013|
|               Japan|                   75.2|2020|
|               Japan|                 75.100|2021|
|               Japan|                   75.1|2019|
|           Singapore|                  75.02|2011|
|               Spain|                   75.0|2020|
|               Japan|                   75.0|2018|
|               Japan|                   74.9|2017|
|           

In [27]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, row_number

# Definir la ventana por la cual se ordenará el dataset y se obtendrá el ranking
window = Window.partitionBy('year').orderBy(desc('Healthy life expectancy'))

# Agregar una columna 'ranking' con el ranking por año de los países por Life Ladder
df_ranking_healty = df_completo_healty.withColumn('rankingHealty', row_number().over(window)).where('rankingHealty == 1')

# Seleccionar las columnas deseadas y mostrar el resultado
df_resultado_healty = df_ranking_healty.select('year', 'Country name', 'Healthy life expectancy')
df_resultado_healty.show()


+----+------------+-----------------------+
|year|Country name|Healthy life expectancy|
+----+------------+-----------------------+
|2005|       Japan|                   73.2|
|2006|   Singapore|                   73.6|
|2007|   Singapore|                   73.9|
|2008|   Singapore|                   74.2|
|2009|   Singapore|                   74.5|
|2010|   Singapore|                   74.8|
|2011|   Singapore|                  75.02|
|2012|       Japan|                  74.16|
|2013|   Singapore|                  75.46|
|2014|   Singapore|                  75.68|
|2015|   Singapore|                   75.9|
|2016|   Singapore|                   76.2|
|2017|   Singapore|                   76.5|
|2018|   Singapore|                   76.8|
|2019|   Singapore|                   77.1|
|2020|       Japan|                   75.2|
|2021|   Singapore|                 76.953|
+----+------------+-----------------------+



In [28]:
# Agrupar por país, contar cuántas veces aparece cada país y ordenar en orden descendente
repeticiones_pais_healty = df_resultado_healty.groupBy('Country name').count().orderBy(('count'))

# Imprimir el resultado
repeticiones_pais_healty.show()


+------------+-----+
|Country name|count|
+------------+-----+
|       Japan|    3|
|   Singapore|   14|
+------------+-----+

